In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pickle
from pprint import pprint

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandapower as pp

from network_utils import (
    create_56bus,
    create_RX_from_net,
    np_triangle_norm,
    read_load_data,
    X_to_ancestors,
    build_tree_from_ancestors)
from run import meta_gen_X_set

In [ ]:
net = create_56bus()

R, X = create_RX_from_net(net, noise=0)  # true R and X

In [ ]:
net = create_56bus()
G_true = pp.topology.create_nxgraph(net)
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape

v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]
Vpar_min = np.min(vpars, axis=0)  # shape [n]
Vpar_max = np.max(vpars, axis=0)  # shape [n]

print(T, n)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 8), tight_layout=True)
pos = nx.nx_agraph.graphviz_layout(G_true, prog='dot')
nx.draw_networkx(G_true, pos=pos, ax=ax, node_color='orange')
ax.set_title('$X^*$')
fig.savefig('scratch/graph_true.png')

In [ ]:
# ancestors = X_to_ancestors(X)
# G = build_tree_from_ancestors(ancestors)
# G = nx.relabel_nodes(G, lambda x: x+1)

In [ ]:
with open('out/CBCproj_noise1.0_perm_norm1.0_seed8_20220211_070643.pkl', 'rb') as f:
    x = pickle.load(f)
print(x.keys())
pprint(x['params'])

In [ ]:
eta = x['params']['eta']
nsamples = x['params']['nsamples']
alpha = x['params']['alpha']
seed = x['params']['seed']
norm_bound = x['params']['norm_bound']

gen_X_set = meta_gen_X_set(norm_bound=norm_bound, X_true=X)
X_init = x['X_init']

In [ ]:
ancestors, _ = X_to_ancestors(X_init)
G_init = build_tree_from_ancestors(ancestors)
G_init = nx.relabel_nodes(G_init, lambda x: x+1)

In [ ]:
from cbc.projection import CBCProjection

sel = CBCProjection(
    eta=eta, n=n, T=T, nsamples=nsamples, alpha=alpha,
    v=vpars[0], gen_X_set=gen_X_set, Vpar=(Vpar_min, Vpar_max),
    X_init=X_init, X_true=X, log=None, seed=seed)
sel.v = x['vs']
sel.q = x['qcs']

In [ ]:
import os
import pickle

path = 'scratch/CBCproj_noise1.0_perm_norm1.0_seed8_20220211_070643_Xhats.pkl'

if os.path.exists(path):
    with open(path, 'rb') as f:
        X_hats = pickle.load(f)

else:
    X_hats = {0: X_init}
    for t in range(1, T):
        sel.add_obs(t)
        if (t+1) % 1000 == 0:
            X_hats[t] = sel.select(t-1).copy()
    X_hats[T] = sel.select(T-1).copy()

    with open(path, 'wb') as f:
        pickle.dump(X_hats, f)

In [ ]:
for t, X_hat in X_hats.items():
    ancestors, Xhat_rounded = X_to_ancestors(X_hat)
    # check_ancestors_completeness(ancestors)

    G_hat = build_tree_from_ancestors(ancestors)
    G_hat = nx.relabel_nodes(G_hat, lambda x: x+1)

    fig, ax = plt.subplots(1, 1, figsize=(5, 8), tight_layout=True)
    pos = nx.nx_agraph.graphviz_layout(G_hat, prog='dot')
    nx.draw_networkx(G_hat, pos=pos, ax=ax, node_color='orange')
    ax.set_title('$\hat{X}, ' + f't={t}$')
    fig.savefig(f'scratch/X_hat_{t:06d}.png')